In [59]:
from bokeh.plotting import figure
from bokeh.io import output_notebook, show
from bokeh.models.tools import HoverTool
from bokeh.models import ColumnDataSource, Range1d, Div, Label
from bokeh.transform import linear_cmap
from bokeh.layouts import row, column
import pandas as pd
import numpy as np
from model_prep import shift_data
from dictionaries import color_dict, title_dict, skill_max, skill_avg

In [2]:
df = pd.read_csv('./data/target_df', index_col=0)

#### Preparing the target dataframe

In [3]:
# Creating a 2019
index_18 = df[df['SEASON']==2018].index
temp_df = df.loc[index_18, ['PLAYER_NAME', 'PLAYER_ID', 'TEAM_ABBREVIATION', 'SEASON']]
temp_df['SEASON'] = 2019
temp_df['TEAM_ABBREVIATION'] = np.nan
for col in df.columns:
    if col not in temp_df.columns:
        temp_df[col] = np.nan
df = pd.concat([temp_df, df], axis=0)
df.reset_index(drop=True, inplace=True)

In [4]:
# Moving the predictions to the following year
for player in df['PLAYER_ID'].unique():
    df[df['PLAYER_ID'] == player]

In [6]:
# Moving the predictions to the following year
target_columns = [col for col in df.columns if 'PRED' in col]
df = shift_data(df=df, on='PLAYER_ID', columns=target_columns, prefix='', amount=-1)

#### Creating the graph

In [7]:
#### FIX THE DICTIONARIES, there should be 1 dictionary (or df) that for each category has all the info (title, stats)
#### AVG performance should be by year (as should max)

In [8]:
def team_colors(df, player, color_dict=color_dict):
    
    # Finding the most colors for the most recent team of the player
    team = df[df['PLAYER_NAME']==player]['TEAM_ABBREVIATION'].iloc[1]
    primary = color_dict['primary'][team]
    secondary = color_dict['secondary'][team]
    
    return primary, secondary

In [204]:
def graph_trends(df, player, skill, color_dict, title_dict, skill_max):
    
    # Creating the player df
    player_index = df[df['PLAYER_NAME']==player].index
    player_df = df.loc[player_index]
    
    # Setting the colors and title
    primary, secondary = team_colors(df, player, color_dict)
    title = title_dict[skill]

    # Creating the plot
    p = figure(plot_height=450, plot_width=450, title=title, tools="wheel_zoom,reset")

    # Saving the plot settings
    if 'RATE' in skill:
        p.yaxis.axis_label = 'Rate'
    elif 'EV' in skill:
        p.yaxis.axis_label = 'Performance'
    p.xaxis.axis_label = 'Season'
    p.xaxis.axis_label_text_font_style = 'normal'
    p.yaxis.axis_label_text_font_style = 'normal'
    p.xaxis.ticker = list(player_df['SEASON'].unique())
    p.outline_line_width = 5
    p.outline_line_alpha = .5
    p.axis.minor_tick_in = -3
    p.axis.minor_tick_out = 6
    p.xaxis.minor_tick_line_color = None
    p.y_range = Range1d(0, skill_max[skill])
    p.toolbar.autohide = True

    # Creating the graphs
    l1 = p.line(player_df['SEASON'], player_df[skill], legend='Actual')
    c1 = p.circle(player_df['SEASON'], player_df[skill], legend='Actual')
    
    l2 = p.line(player_df['SEASON'], player_df['PRED_' + skill], legend='Predictions')
    c2 = p.circle(player_df['SEASON'], player_df['PRED_' + skill], legend='Predictions')

    # Line settings
    l1.glyph.line_width=2
    l1.glyph.line_color=primary
    
    l2.glyph.line_width=2
    l2.glyph.line_color=secondary
    l2.glyph.line_dash=[5,1]
    
    # Legend settings
    if player_df.iloc[1][skill] >= skill_max[skill] - .3:
        p.legend.location = 'bottom_right'
    
    # Circle settings
    c1.glyph.fill_color=primary
    c1.glyph.line_color=primary
    c1.glyph.line_width=2
    c1.glyph.fill_alpha=1
    c1.glyph.size=10

    c2.glyph.fill_color=secondary
    c2.glyph.line_color=secondary
    c2.glyph.line_width=2
    c2.glyph.fill_alpha=1
    c2.glyph.size=10

    return p

In [205]:
def graph_skills(df, player, season, color_dict, title_dict, skill_max, skill_avg):
    
    # Creating the player df
    player_season_index = df[(df['PLAYER_NAME']==player)&(df['SEASON']==season)].index
    player_df = df.loc[player_season_index]
    player_df.fillna(0, inplace=True)
    
    # Setting the colors and title
    primary, secondary = team_colors(df, player, color_dict)
    
    # Extracting the data
    # In the future, the skill max might need to be rethought
    pull_up_rate = max(0, player_df['PULL_UP_RATE'].values[0]/skill_max['PULL_UP_RATE'])
    pull_up_ev = max(0, player_df['PULL_UP_EV'].values[0]/skill_max['PULL_UP_EV'])
    post_up_rate = max(0, player_df['POST_TOUCH_RATE'].values[0]/skill_max['POST_TOUCH_RATE'])
    post_up_ev = max(0, player_df['POST_TOUCH_EV'].values[0]/skill_max['POST_TOUCH_EV'])
    drive_rate = max(0, player_df['DRIVE_RATE'].values[0]/skill_max['DRIVE_RATE'])
    drive_ev = max(0, player_df['DRIVE_EV'].values[0]/skill_max['DRIVE_EV'])
    catch_shoot_rate = max(0, player_df['CATCH_SHOOT_RATE'].values[0]/skill_max['CATCH_SHOOT_RATE'])
    catch_shoot_ev = max(0, player_df['CATCH_SHOOT_EV'].values[0]/skill_max['CATCH_SHOOT_EV'])
    pull_up_avg = skill_avg['PULL_UP_EV']/skill_max['PULL_UP_EV']
    post_up_avg = skill_avg['POST_TOUCH_EV']/skill_max['POST_TOUCH_EV']
    drive_avg = skill_avg['DRIVE_EV']/skill_max['DRIVE_EV']
    catch_shoot_avg = skill_avg['CATCH_SHOOT_EV']/skill_max['CATCH_SHOOT_EV']
    
    
    # Arranging the data
    rates = ColumnDataSource({'x': [0, pull_up_rate, 0, -post_up_rate],
                            'y': [drive_rate, 0, -catch_shoot_rate, 0]})
    EV = ColumnDataSource({'x': [0, pull_up_ev, 0, -post_up_ev],
                           'y': [drive_ev, 0, -catch_shoot_ev, 0]})
    avg_EV = ColumnDataSource({'x': [0, pull_up_avg, 0, -post_up_avg, 0],
                               'y': [drive_avg, 0, -catch_shoot_avg, 0, drive_avg]})
    
    # Creating the plot
    p = figure(plot_height=450, plot_width=450, title=str(season)+' Skills and Tendencies', tools="wheel_zoom,reset")

    # Plot settings
    p.outline_line_width = 5
    p.outline_line_alpha = .5
    p.xaxis.fixed_location = 0
    p.yaxis.fixed_location = 0
    p.xgrid.grid_line_color = None
    p.ygrid.grid_line_color = None
    p.axis.major_tick_line_color = None
    p.axis.minor_tick_line_color = None
    p.axis.ticker = []
    p.axis.bounds = (-1, 1)
    p.y_range = Range1d(-1.2, 1.2)
    p.x_range = Range1d(-1.4, 1.4)
    p.toolbar.autohide = True
    
    # Creating the plot labels
    p.add_layout(Label(x=0, y=1, text='Drive', render_mode='canvas',
                       text_align='center', text_baseline='bottom', text_font_size='10pt'))
    p.add_layout(Label(x=1.05, y=0, text='Pull-Up', render_mode='canvas',
                       text_align='left', text_baseline='middle', text_font_size='10pt'))
    p.add_layout(Label(x=0, y=-1, text='Catch-and-Shoot', render_mode='canvas',
                       text_align='center', text_baseline='top', text_font_size='10pt'))
    p.add_layout(Label(x=-1.05, y=0, text='Post-Up', render_mode='canvas',
                       text_align='right', text_baseline='middle', text_font_size='10pt'))
    
    # Creating the graphs
    p2 = p.patch('x', 'y', source=rates, legend='Rate')
    p1 = p.patch('x', 'y', source=EV, legend='Performance')
    l = p.line('x', 'y', source=avg_EV, legend='Avg Performance')
    
    # Creating the "spider web" grid lines
    p.line([-.5, 0, .5, 0, -.5], [0, .5, 0, -.5, 0], line_alpha=.4, line_color='gray', line_width=.5)
    p.line([-.25, 0, .25, 0, -.25], [0, .25, 0, -.25, 0], line_alpha=.4, line_color='gray', line_width=.5)
    p.line([-.75, 0, .75, 0, -.75], [0, .75, 0, -.75, 0], line_alpha=.4, line_color='gray', line_width=.5)
    p.line([-1, 0, 1, 0, -1], [0, 1, 0, -1, 0], line_alpha=.4, line_color='gray', line_width=.5)
    
    # Patch settings
    p1.glyph.fill_color=primary
    p1.glyph.fill_alpha=.5
    p1.glyph.line_width=2 
    p1.glyph.line_color=primary

    p2.glyph.fill_color=secondary
    p2.glyph.fill_alpha=.5
    p2.glyph.line_width=2 
    p2.glyph.line_color=secondary
    
    # Legend settings
    p.legend.label_text_font_size = '8pt'
    
    # Line settings
    l.glyph.line_alpha=.8
    l.glyph.line_color='black'
    l.glyph.line_width=1
    l.glyph.line_dash=[5,1]

    return p

In [206]:
def double_plot(df, player, season, skill, color_dict, title_dict, skill_max, skill_avg):
    
    # Creating both plots side by side
    p1 = graph_skills(df, player, season, color_dict, title_dict, skill_max, skill_avg)
    p2 = graph_trends(df, player, skill, color_dict, title_dict, skill_max)
    title = Div(text='<h2>'+player+'</h2>')
    plot = column(title, row(p1, p2))
    
    return plot

In [208]:
show(double_plot(df, 'Kevin Durant', 2018, 'PULL_UP_EV', color_dict, title_dict, skill_max, skill_avg))

In [215]:
82*(2**14)/(2**14 + 110**14)

43.593833970662345

In [55]:
show(graph_skills(df, 'Marcus Smart', 2018, color_dict, title_dict, skill_max, skill_avg))

In [16]:
show(graph_trends(df, 'Al Horford', 'POST_TOUCH_RATE', color_dict, title_dict))

In [153]:
columns_of_interest = [col for col in df.columns if ('PRED' not in col) and ('_' in col)]
df[columns_of_interest].describe()

,PLAYER_ID,POST_TOUCH_EV,CATCH_SHOOT_EV,PULL_UP_EV,DRIVE_EV,POST_TOUCH_RATE,CATCH_SHOOT_RATE,PULL_UP_RATE,DRIVE_RATE
count,2.397000e+03,1291.000000,2036.000000,2036.000000,1931.000000,2036.000000,2036.000000,2036.000000,2036.000000
mean,3.106529e+05,0.779468,0.938162,0.723985,0.819514,0.035458,0.108023,0.076243,0.105191
std,4.458900e+05,0.372199,0.288595,0.311625,0.302040,0.056401,0.062138,0.066989,0.091798
min,7.080000e+02,-1.154011,0.000000,0.000000,-1.136581,0.000000,0.000000,0.000000,0.000000
25%,2.011470e+05,0.644148,0.822000,0.616000,0.706065,0.000000,0.063213,0.021710,0.033938
50%,2.023480e+05,0.824177,0.986000,0.782000,0.828932,0.010499,0.103003,0.058396,0.078610
75%,2.034860e+05,1.000000,1.112000,0.888000,0.942495,0.048052,0.148760,0.115590,0.155602
max,1.628021e+06,2.000000,3.000000,3.000000,4.000000,0.418776,0.384615,0.361272,0.578947


In [101]:
master_df = pd.read_csv('./data/master_df', index_col=0)

In [105]:
target_cols = [col for col in title_dict.keys()]

In [114]:
(master_df['CATCH_SHOOT_EV'] * master_df['MIN']).sum()/master_df['MIN'].sum()

0.9698239331368969

In [116]:
avg_dict = {}

In [117]:
for col in target_cols:
    avg_dict[col] = (master_df[col] * master_df['MIN']).sum()/master_df['MIN'].sum()

In [119]:
avg_dict

{'POST_TOUCH_RATE': 0.03978835715950378,
 'POST_TOUCH_EV': 0.5362031339674892,
 'DRIVE_RATE': 0.11624891464574634,
 'DRIVE_EV': 0.8134376186193863,
 'CATCH_SHOOT_RATE': 0.10991854980561748,
 'CATCH_SHOOT_EV': 0.9698239331368969,
 'PULL_UP_RATE': 0.08614674479312547,
 'PULL_UP_EV': 0.7581084463963609}